In [1]:
# ETL
import os
import json
import joblib
import numpy as np
import pandas as pd
from datetime import datetime

# Snowpark functions
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import lit
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION

# Encoding, training, prediction
import xgboost as xgb

#Visualization
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
conn = Session.builder.configs(connection_parameters).create()
conn.sql_simplifier_enabled = True
snowflake_environment = conn.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

The below step is not required if you have all the data in a single schema.  

In [5]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection_GDC_churn.json'))
conn_churn = Session.builder.configs(connection_parameters).create()
conn_churn.sql_simplifier_enabled = True
snowflake_environment = conn_churn.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

## Forecasting customer churn 

### Import customer and subscription data from snowflake

In [ ]:
customers = conn.table('SPOTFLIX.PUBLIC."dim_media_customers"').toPandas()
print(customers.head(2))

subscriptions = conn.table('SPOTFLIX.PUBLIC."fact_media_subscription_events"').toPandas()
print (subscriptions.head(2))

In [ ]:
merged = pd.merge(customers, subscriptions, how ='left', on = 'customer_id')

# Below code checks the following conditions : closedate = eventdate , event = Cancelled Subscription
# merged[(merged['event'] == 'Cancelled Subscription') & (merged['closedate'] != merged['eventdate'])]

# Dealing with bad data - details below
# there are few cancelled subscription dates not mentioned as close date, so correcting them (updating closedate= eventdate when event= Cancelled Subscription	)
merged['closedate'] = np.where(merged['event'] == 'Cancelled Subscription', merged['eventdate'], merged['closedate'])

# dropping duplicates
merged.drop_duplicates(subset='customer_id', inplace=True)
print (merged.head(2))

In [8]:
customers_selected = merged[['customer_id', 'opendate', 'closedate']]
print( customers_selected.head() )

   customer_id    opendate   closedate
0            1  2018-03-09        None
1            2  2017-10-18        None
2            3  2018-03-03  2019-04-23
3            4  2018-03-25  2018-07-30
4            5  2017-07-17  2019-04-24


In [ ]:
# Convert 'opendate' and 'closedate' to datetime format
customers_selected['opendate'] = pd.to_datetime(customers_selected['opendate'])
customers_selected['closedate'] = pd.to_datetime(customers_selected['closedate'])

# Calculate the 'days' column without using lambda
customers_selected['No_days_on_platform'] = (customers_selected['closedate'] - customers_selected['opendate']).dt.days.fillna((datetime(2019, 7, 26) - customers_selected['opendate']).dt.days)

# Create the 'churn' column
customers_selected['churn'] = customers_selected['closedate'].notna().astype(int)

customers_selected.drop(['opendate', 'closedate'],axis =1, inplace =True )

# Display the modified DataFrame
customers_selected.head(2)


### Processing Fact media show event data

In [10]:
show_events = conn.table('SPOTFLIX.PUBLIC."fact_media_show_events"').toPandas()
show_events.drop(['record_id','show_id', 'viewtime','episode_id','viewruntimepct'], axis=1, inplace=True)
# Convert 'viewdate' to datetime
show_events['viewdate'] = pd.to_datetime(show_events['viewdate'])
customer_engagement = pd.merge(show_events,customers_selected, how = 'left', on = 'customer_id')
customer_engagement.head(2)

,customer_id,viewdate,viewruntime,No_days_on_platform,churn
0,23927,2018-07-22,24,108.0,1
1,23927,2018-07-22,24,108.0,1


### Using Date Reference Table 

In [11]:
date_ref = conn.table('SPOTFLIX.PUBLIC."dim_media_date_ref"').toPandas()
date_ref.drop('date_id', axis = 1, inplace = True) 
date_ref['date'] = pd.to_datetime(date_ref['date'])
customer_engagement['viewdate'] = pd.to_datetime(customer_engagement['viewdate'])
date_wise_merged = pd.merge(date_ref, customer_engagement, left_on='date', right_on='viewdate', how='right')
date_wise_merged.drop(['date','No_days_on_platform'], axis = 1, inplace =True	)
date_wise_merged.head(2)

,customer_id,viewdate,viewruntime,churn
0,23927,2018-07-22,24,1
1,23927,2018-07-22,24,1


In [12]:
date_wise_merged['viewdate'] = pd.to_datetime(date_wise_merged['viewdate'])

grouped_date_wise = date_wise_merged.groupby(['viewdate', 'customer_id']).agg({
    'viewruntime': 'sum'
}).reset_index()

grouped_date_wise.head(2)

,viewdate,customer_id,viewruntime
0,2018-06-17,577,58
1,2018-06-17,579,58


In [13]:
# Helper function that generates information required for model training
#This function uses date array from inout data to build necessary columns for model training
def date_conv(date_array):
    date_array_df = pd.DataFrame(date_array.values, columns = ['viewdate'])
    date_array_df['viewdate'] = pd.to_datetime(date_array_df['viewdate'])
    date_array_df['day_of_year'] = date_array_df['viewdate'].dt.dayofyear
    date_array_df['day_of_month'] = date_array_df['viewdate'].dt.day
    date_array_df['day_of_week'] = date_array_df['viewdate'].dt.dayofweek  # Monday=0, Sunday=6
    date_array_df['month'] = date_array_df['viewdate'].dt.month
    date_array_df['quarter'] = date_array_df['viewdate'].dt.quarter
    date_array_df['year'] = date_array_df['viewdate'].dt.year
    return date_array_df

## Training

In [14]:
# Creates training dataset at specified granularity - eg: customer , show, date level
def make_dataset(df, date_col, customer_id_col, agg_col, agg_func):
    grp_df = df.groupby([date_col, customer_id_col]).agg({
        agg_col: agg_func,
    }).reset_index()
    pivot_df = grp_df.pivot(index=date_col, columns=customer_id_col, values=agg_col)

    itr = len(pivot_df.columns)
    df_list = []
    keys = pivot_df.columns.to_list()

    for i in range(itr):
        col = pivot_df.columns[i]
        df1 = pd.DataFrame(pivot_df[col])

        # Drop NaN values using the aggregated column name
        df1.dropna(subset=[col], inplace=True)

        training_data = date_conv(df1.index)
        training_data = training_data.set_index(date_col)
        training_data[agg_col] = df1[col]
        df_list.append(training_data)

    return df_list, keys


In [15]:
make_dataset_test, keys = make_dataset(date_wise_merged, 'viewdate', 'customer_id', 'viewruntime', 'sum')

In [19]:
# Time series forecasting model using xgboost
# Forecasts data for multiple shows, customers, dates etc.,
def train_xgboost_models(category_data_list, keys,agg_col,folder_path):
    fi = []
    fn = []
    for i in range(len(category_data_list)):
        # for category, data in category_data.items():
            # Convert the date index

        print (category_data_list[i])

        training_data = date_conv(category_data_list[i].index)
        print(training_data)

        # Set the index to 'Transaction_Date'
        # training_data = training_data.set_index('Transaction_Date')
        feature_cols = category_data_list[i].columns[:-1]
        print(feature_cols)

        # Set the 'Sales_Quantity' column to the corresponding category
        # training_data['Sales_Quantity'] = keys

        # Select features for training the model
        # feature_cols = ['day_of_year', 'day_of_month', 'day_of_week', 'month', 'quarter', 'year']
        X = category_data_list[i].drop(agg_col, axis=1)
        y_quantity = category_data_list[i][agg_col]

        # Define the best hyperparameters obtained from hyperparameter tuning
        best_params = {
            'max_depth': 6,
            'eta': 0.3,
            'gamma': 0,
            'subsample': 1
        }

        # Initialize XGBoost model with the best hyperparameters
        model = xgb.XGBRegressor(n_estimators=200, n_jobs=1,
                                    max_depth=best_params['max_depth'],
                                    eta=best_params['eta'],
                                    gamma=best_params['gamma'],
                                    subsample=best_params['subsample']
                                    )

        # Fit the model
        model.fit(X, y_quantity)


        # folder_path = '/Users/mohammed.arshad/Downloads/Models'
        filename = f'{folder_path}/xgb_classifier_{keys[i]}.sav'
        joblib.dump(model, filename)


        # Feature importance
        feat_importance = pd.DataFrame(
            model.feature_importances_, feature_cols, columns=["FeatImportance"]
        ).to_dict()

        fi.append(feat_importance)
        fn.append(filename)

    return fi, fn



In [ ]:
agg_col = 'viewruntime'
folder_path = 'Models'
feature_Imp, models = train_xgboost_models(make_dataset_test[0:100], keys,agg_col, folder_path)

#### Sampling the data for faster analysis

In [ ]:
data_trained = date_wise_merged[date_wise_merged['customer_id']<=100]
data_trained .drop('churn',inplace = True, axis =1)
data_trained

In [22]:
# Method used to forecast data using the trained models at customer level
def forecast_and_add_to_dataframe(model_folder,data_trained):
    # Check if the model folder exists
    if not os.path.exists(model_folder):
        raise FileNotFoundError(f"The model folder '{model_folder}' does not exist.")
    
    forecast_end_date = '2021-01-31'
    predicted_df = pd.DataFrame()
    final_df= pd.DataFrame()


    # Loop through each file in the model folder
    for filename in os.listdir(model_folder):
        # Assuming models are saved with a specific extension, e.g., '.pkl'
        if filename.endswith('.sav'):
            # Construct the full path to the model file
            model_path = os.path.join(model_folder, filename)
            # print(model_path)
            # Load the model
            model = joblib.load(model_path)  # Adjust this for other model loading methods



# one line to get customer_id from filename  - os.path.splitext(filename.split("_")[-1])[0]
# slice the data_trained using customer id  and sort by view date .iloc[-1,'viewdate']
           
            customer_id = int(os.path.splitext(filename.split("_")[-1])[0])
            print(customer_id)

            # Slice data_trained and sort by 'viewdate' for the specific customer_id
            last_available_date = data_trained[data_trained['customer_id'] == customer_id].sort_values('viewdate').iloc[-1]['viewdate']
            # print (last_available_date)

            # Display the last available date for the specified customer_id
            # print(f"Last available date for customer {customer_id}: {last_available_date}")


            forecast_start_date = last_available_date + pd.DateOffset(days=1)
            # print (forecast_start_date)
            forecast_start_date=pd.to_datetime(forecast_start_date)
            forecast_end_date=pd.to_datetime(forecast_end_date)
            date_array = pd.date_range(forecast_start_date,forecast_end_date, freq='D')
            forecast_data = date_conv(date_array).set_index('viewdate')
            # print (forecast_data)

            # Make predictions on the new data
            predictions = model.predict(forecast_data)

            # print (predictions)

            print(len(forecast_data))
            print(len(predictions))


            # Create a DataFrame for the predicted data
            predicted_df = pd.DataFrame({
                'customer_id': [int(os.path.splitext(filename.split("_")[-1])[0])] * len(predictions),
                'viewruntime': predictions,
                'viewdate': forecast_data.index,  # Assuming 'forecast_data' has 'viewdate' as an index
            })

            # Concatenate the predicted data to the final_df
            frames = [final_df, predicted_df]
            final_df = pd.concat(frames, ignore_index=True)

        # Sort the final_df by 'viewdate' if needed
        final_df.sort_values('viewdate', inplace=True)


    return final_df



#### Using created models to run forecast

In [ ]:
# Example usage:
model_folder_path = 'Models'

# Assuming forecast_data is your DataFrame with the data for forecasting
forecast_data = forecast_and_add_to_dataframe(model_folder_path,data_trained)
forecast_data.reset_index(drop=True, inplace=True)
forecasted_churn = pd.concat([data_trained, forecast_data], ignore_index=True)
forecasted_churn

### Writing forecasted data to Snowflake 

In [24]:
forecasted_churn['viewdate'] = forecasted_churn['viewdate'].dt.date
conn_churn.createDataFrame(forecasted_churn).write.mode('overwrite').save_as_table('forecasted_viewtime_100_customers')